#Feature Extraction

The aim of this jupyter notebook is to build upon the data file extraction made by Amanda. The main idea is to use pre
existing libraries susch as *scikit-learn*. The main goal is to learn the basics, and train a machine learning model
quickly and easily.

In [2]:
# First we will extract the data, and save it to a csv.
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [3]:
train_a = pd.read_parquet('A/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')

In [4]:
print(train_a.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34085 entries, 0 to 34084
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   time            34085 non-null  datetime64[us]
 1   pv_measurement  34085 non-null  float64       
dtypes: datetime64[us](1), float64(1)
memory usage: 532.7 KB


In [5]:
X_train_estimated_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17576 entries, 0 to 17575
Data columns (total 47 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date_calc                       17576 non-null  datetime64[us]
 1   date_forecast                   17576 non-null  datetime64[us]
 2   absolute_humidity_2m:gm3        17576 non-null  float32       
 3   air_density_2m:kgm3             17576 non-null  float32       
 4   ceiling_height_agl:m            13657 non-null  float32       
 5   clear_sky_energy_1h:J           17576 non-null  float32       
 6   clear_sky_rad:W                 17576 non-null  float32       
 7   cloud_base_agl:m                15482 non-null  float32       
 8   dew_or_rime:idx                 17576 non-null  float32       
 9   dew_point_2m:K                  17576 non-null  float32       
 10  diffuse_rad:W                   17576 non-null  float32       
 11  di

In [6]:
def data_length_matching(train: pd.DataFrame, obs: pd.DataFrame)-> tuple[pd.DataFrame, pd.DataFrame]:
    """
    This function is intended to ensure that both the training data and
    the observed data are sorted, and contain the same number of entries. 
    """

    # Cut the data frames so that their date match.
    obs_feature_test = obs[obs['date_forecast'].isin(train['time'])].sort_values(by=['date_forecast']) # sortert etter datao
    
    # If only one of them has the date ensure that the other also has the same sorting.
    train_feature_test = train[train['time'].isin(obs['date_forecast'])].sort_values(by=['time']) # sortert etter datao

    # Would not the rest ensure this?
    print('If True same length and time stamps')
    print(len(obs_feature_test) == len(train_feature_test))
    print(len(obs_feature_test), len(train_feature_test))

    return train_feature_test, obs_feature_test

def dt64_to_float(dt64):
     year = dt64.astype('M8[Y]')
     days = (dt64 - year).astype('timedelta64[D]')
     year_next = year + np.timedelta64(1, 'Y')
     days_of_year = (year_next.astype('M8[D]') - year.astype('M8[D]')).astype('timedelta64[D]')
     dt_float = 1970 + year.astype(float) + days / (days_of_year)
     return dt_float

X_train_observed_a = X_train_observed_a.drop("date_calc", axis='columns')
X_total = pd.concat([X_train_estimated_a, X_train_observed_a], axis = 0)

y, X = data_length_matching(train_a, X_total)

y['time'] = dt64_to_float(y['time'].to_numpy())

X['date_forecast'] = dt64_to_float(X['date_forecast'].to_numpy())
X['date_calc'] = dt64_to_float(X['date_calc'].to_numpy())

X = X.fillna(0)
y = y.drop('time', axis=1)

X_new = SelectKBest(f_regression, k=5).fit_transform(X, y)
print("This is x-new")
print(X_new)

#+====================================================================================================================+
#| Additional idea: Notice that time of day parameters will probably be provided... Fitting this to a location might  |
#| give us an edge needed to fit a better algorithm for each location...                                              |
#+====================================================================================================================+



# Use some of scikit learn feature extraction functionality.

# VarianceThreshold

# SelectKBest

# Tree-based feature selection

# Try implementing this into: Pipeline (1.13.6.)


KeyError: "['date_calc'] not found in axis"